In [ ]:
import pandas as pd
import numpy as np
from BeyondBlooms2024.config import name_dict
from BeyondBlooms2024.config_file import (ABUNDANCES_FILE, CCMN_CON_MAP_PATH, CON_LOUVAIN_META_PATH,CON_LOUVAIN_NETWORK_PATH, ENRICH
,NUM_PERMUTATIONS, NUM_SAMPLES, NUM_CORES, METADATA_FILE, PRUNED_PVAL_CCMN_PATH,PVAL_CCMN_PATH,ENRICHED_META_PATH, RANDOM_PVAL_CCMN_PATH)

# Env Data

In [ ]:
df = pd.read_csv(METADATA_FILE, sep=";")
df_env_F4 = df
df_env_F4 = df_env_F4.set_index("date")
df_env_F4.index = pd.to_datetime(df_env_F4.index)
df_env_F4.columns#'PAR_satellite'
print(len(df_env_F4))
#df_env_F4 = df_env_F4.loc['2016-08-01':'2020-07-31']
print(df_env_F4)
cols =['MLD', 'PAR_satellite', 'temp', "sal", "PW_frac", "O2_conc","depth"]# df_env_F4.columns#
# corr with only one mooring ASV
df_env = df_env_F4[cols]
window_size = 8  # You can adjust this as needed
df_env = df_env.reset_index()
for i in range(len(df_env)):
    if pd.isna(df_env.at[i, 'O2_conc']):
        # Calculate the rolling mean for the neighborhood around the NaN value
        start = max(0, i - (window_size // 2))
        end = min(len(df_env), i + (window_size // 2) + 1)
        neighborhood_mean = df_env['O2_conc'][start:end].mean()

        # Replace NaN with the neighborhood mean
        df_env.at[i, 'O2_conc'] = neighborhood_mean
# Now, df_env will have NaN values in the "pH" column replaced with rolling mean values around them
print(df_env)
df_env.set_index("date", inplace=True)
#print(df_env)
df_env.to_csv("/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper_Submission/data/Env_data.csv", sep=";")

In [ ]:
df_env_time = df_env[["MLD"]]
df_env_time

# Taxa and Labels

In [ ]:
df_taxa = pd.read_csv(ENRICH, sep =',', engine="python")
df_taxa.shape

In [ ]:
def get_dict(col = "Class"):
    df_taxa_temp =df_taxa[["Nodes",col]]
    df_taxa_temp.set_index("Nodes", inplace=True)
    d_ = df_taxa_temp.to_dict()[col]
    return d_

# Abundance Table

In [ ]:
df_abundance = pd.read_csv(ABUNDANCES_FILE,sep=";")
df_abundance["Unnamed: 0"]= pd.to_datetime(df_abundance["Unnamed: 0"])
df_abundance.set_index("Unnamed: 0",inplace=True)
# Hellinger Normalize
#df_sqrt = np.sqrt(df_abundance)
#row_norms = np.linalg.norm(df_sqrt, axis=1)
#df_normalized = df_sqrt.div(row_norms, axis=0)
#df_abundance = df_normalized
df_abu= df_abundance.join(df_env_time, how="inner")
df_abu.drop(columns=["MLD"],inplace=True)
df_abundance=df_abu
df_abu["count"]=1
df_abu = df_abu[["count"]]

In [ ]:
print(df_env.shape)
df_env_ =df_env.join(df_abu, how="inner")
print(df_env_.shape)
df_env_.drop(columns=["count"],inplace=True)
df_env_.to_csv("/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper_Submission/data/Raw_Env_data.csv", sep=";")
print(df_env_.shape)

In [ ]:
df_env_

# Create Clusters

In [ ]:
clu_list = df_taxa["cluster_names"].unique().tolist()
clu_list.sort()
clu_list

In [ ]:
from BeyondBlooms2024.config import name_dict
# ToDo Name
for clu in clu_list:
    print(clu)
    temp_df = df_abundance[df_taxa[df_taxa["cluster_names"]==clu]["Nodes"].tolist()]
    #print(temp_df.shape, clu)
    #print(temp_df.head(2))
    temp_df.to_csv(f"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper_Submission/data/Raw_Ela_Cluster_{clu}.csv", sep=";")
    print(f"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper_Submission/data/Raw_Ela_Cluster_{clu}.csv")

In [ ]:
df_taxa.columns

In [ ]:
cluster_nos = df_taxa[["Nodes","max_abundance_month","cluster_names"]]

In [ ]:
cluster_nos

In [ ]:
["/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_01-F_-09-|-10-|-11-_tr0.02.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_02-F_-09-|-10-|-11-_tr0.02.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_03-L_-12-|-01-|-02-_tr0.002.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_04-L_-12-|-01-|-02-_tr0.002.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_05-L_-12-|-01-|-02-_tr0.002.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_06-M_-03-|-04-|-05-_tr0.02.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_07-M_-03-|-04-|-05-_tr0.02.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_08-M_-03-|-04-|-05-_tr0.02.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_09-H_-06-|-07-|-08-_tr0.02.csv",
"/Users/raphaelkronberg/PycharmProjects/ELLEN/CCMN_Paper/tables/woSeason/Raw_NMDS_coordinates_for_REla_with_env__mod_withoutSeason_10-H_-06-|-07-|-08-_tr0.02.csv",
]